In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"

referer = "https://zigbang.com/home/oneroom/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}
sad_words = [ '대출x', '대출불가', '전세 안됩']

In [4]:
def export_items(items):
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address1',
        'address2',
        'address3',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=4)
    
    wish = '대출'
    for item in items:
        item = item['item']
        if wish in item['title'] or wish in item['description']:
            plz_flag = True
            for word in sad_words:
                if word in item['description']:
                    plz_flag = False
                    break
            if not plz_flag:
                continue
            item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
            if maximum_updated < datetime.datetime.now() - item_updated:
                continue
            item_id = item['id']
            url = magic_fstring_function(item_view_url)
            item['url'] = str(url)
            new_item = {}
            for fieldname in fieldnames:
                new_item[fieldname] = item[fieldname]
            filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/oneroom/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    deposit_gteq: int = 0,
    deposit_lteq: int = 8000,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wydjr",
    rent_gteq: int = 0,
    sales_type_in: str = "전세",
    service_type_eq: str = "원룸",
):
    resp = requests.get(
        url=item_list_api,
        params={
            "deposit_gteq": deposit_gteq,
            "deposit_lteq": deposit_lteq,
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "sales_type_in": sales_type_in,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}


In [8]:
geos_sinrim = [
    'wydm0',
    'wydm1',
    'wydm2',
   'wydm3',
] 

for geo in geos_sinrim:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [10]:
len(items)

191

In [11]:
result = describe_room_list(items)

In [12]:
len(result)

184

In [13]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '2층',
  'address1': '서울시 관악구 신림동',
  'address2': '90-82',
  'address3': None,
  'agent_address1': '서울특별시 관악구 봉천로 391, 1층 101-1호(봉천동, 두산아파트 상가1동)',
  'agent_comment': '',
  'agent_email': 'zeremien@naver.com',
  'agent_lat': 37.4834714149035,
  'agent_lng': 126.943250339993,
  'agent_local1': '서울시',
  'agent_local2': '관악구',
  'agent_mobile': '010-3894-9420',
  'agent_name': '다와부동산공인중개사사무소(전미라)',
  'agent_no': 67990,
  'agent_phone': '02-2135-2474',
  'bjd_code': '',
  'bonbun_code': '90',
  'bubun_code': '82',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 4000,
  'description': '💙시진 보시면 아시겠지만 채광,통풍 아주 굿~\r\n'
                 '\r\n'
                 '💙지상층에 이런 가격이 ~너무 깔끔한 원룸이라서 보시면\r\n'
                 '       반하실꺼에요~\r\n'
                 '💙관리비 올포함~\r\n'
                 '\r\n'
                 '💙직장인,대학생,학원생분들께 추천 드립니다~\r\n'
                 '     (워낙 조용한 동네라 만족 하실꺼에요~)\r\n'
                 '💙나가시면(편의점,버스정류장,맛집,시장,쇼핑몰,세탁소,까페,병원,약국등~)\r\

In [14]:
filtered_result = export_items(result)

In [15]:
len(filtered_result)


23

In [16]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [17]:
geo_based_dict = {}

In [18]:

for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)

In [19]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ '_floor': '3층',
  'address': '서울시 관악구 신림동 87-61 ',
  'address1': '서울시 관악구 신림동',
  'address2': '87-61',
  'address3': None,
  'agent_mobile': '010-8980-3144',
  'agent_name': '119부동산공인중개사사무소(유미선)',
  'deposit': 5000,
  'description': '💞 매물 정보\n'
                 '  \n'
                 '신림역 전세대출 가능한 오픈형 원룸 \n'
                 '대로변 위치라 늦게 퇴근 하시는 분 \n'
                 '치안 걱정 없이 안심귀가로 걱정 NO\n'
                 '인근 풍부한 먹거리로 간단하게 이용 가능\n'
                 '다양한 편의시설 마트 , 병원, 약국, 은행, \n'
                 '편의점, 세탁소, 편하게 이용!!\n'
                 '인근 산책 할 수 있는 공원 있어서\n'
                 '산책 하고 내몸 지키기!!\n'
                 '\n'
                 '💞 방 체크 \n'
                 '\n'
                 '👉교통\n'
                 '     버스정류장 도보 3분 거리,  신림역 도보 5분 거리 \n'
                 '\n'
                 '👉 기본 옵션\n'
                 '     세탁기, 에어컨, 인덕션, 냉장고, 전자도어락\n'
                 '     침대는 세입자가 꾸민 거라 비 옵션 입니다. \n'
                 '\n'
                 '👉 관리비\n'
                 '    관리비10만원(인터넷, 

In [20]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [21]:
with open('crawling_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [24]:
"""
create table test4(
        no int not null auto_increment,
        id varchar(20) not null,
        deposit varchar(100),
        agent_name varchar(200),
        agent_mobile varchar(200),
        floor varchar(100),
        address varchar(100),
        size varchar(100),
        size_m2 varchar(100),
        status varchar(100),
        room_type varchar(100),
        manage_cost_inc varchar(100),
        url varchar(100),
        upddt varchar(50),
        primary key(no)
        );

"""

'\ncreate table test4(\n        no int not null auto_increment,\n        id varchar(20) not null,\n        deposit varchar(100),\n        agent_name varchar(200),\n        agent_mobile varchar(200),\n        floor varchar(100),\n        address varchar(100),\n        size varchar(100),\n        size_m2 varchar(100),\n        status varchar(100),\n        room_type varchar(100),\n        manage_cost_inc varchar(100),\n        url varchar(100),\n        upddt varchar(50),\n        primary key(no)\n        );\n\n'

In [25]:
import pymysql
import csv

conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='python_db', charset='utf8')
curs = conn.cursor()
sql = "insert into test4 (id,deposit,agent_name,agent_mobile,floor,address,size,size_m2,status,room_type,manage_cost_inc,url,upddt) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, now())"
f = open('crawling_data.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

#title, description은 이모티콘 인식못하는 에러 -> 일단 제외
for line in rd:
    curs.execute(sql, (line[0], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13]))

conn.commit()
conn.close()
f.close()

ProgrammingError: (1146, "Table 'python_db.test4' doesn't exist")